# nnUNet, extraction of whole vertebra and vertebral body images

In [ ]:
import os
import json
import shutil
import nibabel as nib
import numpy as np

In [ ]:
!pip install nnunetv2

In [ ]:
!pip install -U triton

In [ ]:
# Define dataset details
task_name = 'Dataset001_spine'
base_dir = 'PATH/nnUNet_raw_data'  # Replace 'PATH' with the actual base directory
task_dir = os.path.join(base_dir, task_name)
imagesTs_dir = os.path.join(task_dir, 'imagesTs')

# Create necessary directories
os.makedirs(imagesTs_dir, exist_ok=True)

In [ ]:
# Define environment variables for nnUNet's required paths
os.environ['nnUNet_raw'] = 'PATH/nnUNet_raw_data'
os.environ['nnUNet_preprocessed'] = 'PATH/nnUNet_preprocessed'
os.environ['nnUNet_results'] = 'PATH/nnUNet_results'

# Ensure directories exist by creating them if they don't
os.makedirs(os.environ['nnUNet_raw'], exist_ok=True)
os.makedirs(os.environ['nnUNet_preprocessed'], exist_ok=True)
os.makedirs(os.environ['nnUNet_results'], exist_ok=True)

In [ ]:
# Display available options for nnUNet's best model selection
!nnUNetv2_find_best_configuration -h

In [ ]:
# Find the best model configuration using a specific trainer and plan
!nnUNetv2_find_best_configuration 001 -tr nnUNetTrainer_500epochs -p nnUNetResEncUNetLPlans

In [ ]:
# Display help menu for the nnUNet inference command
!nnUNetv2_predict -h

In [ ]:
# Run inference on test images using 2D model configuration
!nnUNetv2_predict -d Dataset001_spine -i "PATH/nnUNet_raw_data/Dataset001_spine/imagesTs" -o "PATH1" -f  0 1 2 3 4 -tr nnUNetTrainer_500epochs -c 2d -p nnUNetResEncUNetLPlans --save_probabilities

In [ ]:
# Run inference on test images using 3D full-resolution model configuration
!nnUNetv2_predict -d Dataset001_spine -i "PATH/nnUNet_raw_data/Dataset001_spine/imagesTs" -o "PATH2" -f  0 1 2 3 4 -tr nnUNetTrainer_500epochs -c 3d_fullres -p nnUNetResEncUNetLPlans --save_probabilities

In [ ]:
# Combine the outputs of the 2D and 3D full-resolution models to improve accuracy
!nnUNetv2_ensemble -i "PATH1" "PATH2" -o "PATH3" -np 8

### Create and save extracted images in NIfTI format

In [ ]:
# Define the paths to input and output folders
folder1 = "PATH/nnUNet_raw_data/Dataset001_spine/imagesTs"
folder2 = "PATH3"
output_folder = "PATH4"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Get sorted lists of files from both folders
files1 = sorted([f for f in os.listdir(folder1) if f.endswith('.nii.gz')])
files2 = sorted([f for f in os.listdir(folder2) if f.endswith('.nii.gz')])

# Ensure that both folders contain the same number of files
if len(files1) != len(files2):
    raise ValueError("The two folders must contain the same number of files!")

# Process each pair of files
for file1, file2 in zip(files1, files2):
    # Load the NIfTI files
    img1 = nib.load(os.path.join(folder1, file1))
    img2 = nib.load(os.path.join(folder2, file2))

    # Extract the data arrays
    data1 = img1.get_fdata()
    data2 = img2.get_fdata()

    # Multiply the data arrays to apply the segmentation mask
    multiplied_data = data1 * data2

    # Rotate the resulting data array 90 degrees for proper alignment
    rotated_data = np.rot90(multiplied_data)

    # Create a new NIfTI image with the rotated data
    new_img = nib.Nifti1Image(rotated_data, affine=img1.affine, header=img1.header)

    # Save the new image to the output folder
    output_path = os.path.join(output_folder, f"multiplied_rotated_{file1}")
    nib.save(new_img, output_path)

    print(f"Processed and saved: {output_path}")
